# Test if MendelImpute can impute untyped SNPs

In [2]:
using Revise
using VCFTools
using MendelImpute
using GeneticVariation
using Random
using StatsBase

┌ Warning: no Manifest.toml file found, static paths used
└ @ Revise /Users/biona001/.julia/packages/Revise/439di/src/Revise.jl:1108
┌ Info: Precompiling MendelImpute [e47305d1-6a61-5370-bc5d-77554d143183]
└ @ Base loading.jl:1273


## Generate subset of markers for prephasing

In [2]:
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
function filter_and_mask()
    for chr in [22]
        # filter chromosome data for unique snps
#         data = "../raw/ALL.chr$chr.phase3_v5.shapeit2_mvncall_integrated.noSingleton.genotypes.vcf.gz"
#         full_record_index = .!find_duplicate_marker(data)
#         @time VCFTools.filter(data, full_record_index, 1:nsamples(data), 
#             des = "chr$chr.uniqueSNPs.vcf.gz")

        # summarize data
        total_snps, samples, _, _, _, maf_by_record, _ = gtstats("chr$chr.uniqueSNPs.vcf.gz")

        # generate target file with 250 samples and 100k snps with maf>0.05
        n = 250
        p = 100000
        record_idx = falses(total_snps)
        large_maf = findall(x -> x > 0.05, maf_by_record)  
        shuffle!(large_maf)
        record_idx[large_maf[1:p]] .= true
        sample_idx = falses(samples)
        sample_idx[1:n] .= true
        shuffle!(sample_idx)
        @time VCFTools.filter("chr$chr.uniqueSNPs.vcf.gz", record_idx, sample_idx, 
            des = "target.chr$chr.typedOnly.vcf.gz")

        # generate target panel with all snps
        @time VCFTools.filter("chr$chr.uniqueSNPs.vcf.gz", 
            1:total_snps, sample_idx, des = "target.chr$chr.full.vcf.gz")

        # also generate reference panel without target samples
        @time VCFTools.filter("chr$chr.uniqueSNPs.vcf.gz", 
            1:total_snps, .!sample_idx, des = "ref.chr$chr.excludeTarget.vcf.gz")

        # unphase and mask 1% entries in target file
        masks = falses(p, n)
        missingprop = 0.01
        for j in 1:n, i in 1:p
            rand() < missingprop && (masks[i, j] = true)
        end
        @time mask_gt("target.chr$chr.typedOnly.vcf.gz", masks, 
            des="target.chr$chr.typedOnly.masked.vcf.gz", unphase=true)

        # generate subset of reference file that matches target file
        @time conformgt_by_pos("ref.chr$chr.excludeTarget.vcf.gz", 
            "target.chr$(chr).typedOnly.masked.vcf.gz", 
            "chr$chr.aligned", "$chr", 1:typemax(Int))
        if nrecords("chr$chr.aligned.tgt.vcf.gz") == p
            rm("chr$chr.aligned.tgt.vcf.gz", force=true) # perfect match
        else
            error("target file has SNPs not matching in reference file! Shouldn't happen!")
        end
        mv("chr22.aligned.ref.vcf.gz", "ref.chr22.aligned.vcf.gz", force=true)
    end 
end
Random.seed!(2020)
@time filter_and_mask()

Progress: 100%|█████████████████████████████████████████| Time: 0:08:32


400.374871 seconds (4.98 G allocations: 468.120 GiB, 9.98% gc time)
472.964733 seconds (5.67 G allocations: 522.326 GiB, 11.03% gc time)
909.764993 seconds (12.13 G allocations: 935.917 GiB, 13.36% gc time)
 13.910180 seconds (76.61 M allocations: 8.666 GiB, 6.13% gc time)


┌ Info: Match target POS to reference POS
└ @ VCFTools /Users/biona001/.julia/dev/VCFTools/src/conformgt.jl:172
Progress: 100%|█████████████████████████████████████████| Time: 0:09:57


616.684922 seconds (7.23 G allocations: 672.405 GiB, 10.84% gc time)


┌ Info: 100000 records are matched
└ @ VCFTools /Users/biona001/.julia/dev/VCFTools/src/conformgt.jl:239


ArgumentError: ArgumentError: 'ref.chr22.aligned.vcf.gz' exists. `force=true` is required to remove 'ref.chr22.aligned.vcf.gz' before moving.

### Missing rate

In typed markers, 1% of data is missing at random. In addition, only 84% of all markers are not typed (i.e. systematically missing). 

In [4]:
tgtfile = "target.chr22.typedOnly.masked.vcf.gz"
reffile = "ref.chr22.excludeTarget.vcf.gz"
missing_rate = 1 - nrecords(tgtfile) / nrecords(reffile)

0.844946576342879

# MendelImpute on untyped markers with dp

In [2]:
Threads.nthreads()

8

In [3]:
tgtfile = "target.chr22.typedOnly.masked.vcf.gz"
reffile = "ref.chr22.excludeTarget.vcf.gz"
reffile_aligned = "ref.chr22.aligned.vcf.gz"
X_typedOnly_complete = "target.chr22.typedOnly.vcf.gz"
X_full_complete = "target.chr22.full.vcf.gz"
@show nrecords(tgtfile), nsamples(tgtfile)
@show nrecords(reffile), nsamples(reffile)
@show nrecords(reffile_aligned), nsamples(reffile_aligned)
@show nrecords(X_typedOnly_complete), nsamples(X_typedOnly_complete)
@show nrecords(X_full_complete), nsamples(X_full_complete)

(nrecords(tgtfile), nsamples(tgtfile)) = (100000, 250)
(nrecords(reffile), nsamples(reffile)) = (644939, 2254)
(nrecords(reffile_aligned), nsamples(reffile_aligned)) = (100000, 2254)
(nrecords(X_typedOnly_complete), nsamples(X_typedOnly_complete)) = (100000, 250)
(nrecords(X_full_complete), nsamples(X_full_complete)) = (644939, 250)


(644939, 250)

In [3]:
# dp method using incomplete ref panel
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
Random.seed!(2020)
function run()
    X_complete = convert_gt(Float32, "target.chr22.full.vcf.gz")
    n, p = size(X_complete)
    chr = 22
    for width in [500, 1000, 2000]
        println("Imputing typed + untyped SNPs with dynamic programming, width = $width")
        tgtfile = "target.chr$chr.typedOnly.masked.vcf.gz"
        reffile = "ref.chr$chr.excludeTarget.vcf.gz"
        outfile = "mendel.imputed.dp$width.vcf.gz"
        @time phase(tgtfile, reffile, impute=true, 
            outfile = outfile, width = width, fast_method=false)
        X_mendel = convert_gt(Float32, outfile)
        println("error overall = $(sum(X_mendel .!= X_complete) / n / p) \n")
    end
end
run()

Imputing typed + untyped SNPs with dynamic programming, width = 500


Importing genotype file...100%|█████████████████████████| Time: 0:00:05
Importing reference haplotype files...100%|█████████████| Time: 0:03:13
Computing optimal haplotype pairs...100%|███████████████| Time: 0:03:04
Merging breakpoints...100%|█████████████████████████████| Time: 0:01:14
Writing to file...100%|█████████████████████████████████| Time: 0:00:17


506.273613 seconds (3.07 G allocations: 288.421 GiB, 8.00% gc time)
error overall = 0.004045622919376871 

Imputing typed + untyped SNPs with dynamic programming, width = 1000


Importing genotype file...100%|█████████████████████████| Time: 0:00:05
Importing reference haplotype files...100%|█████████████| Time: 0:03:15
Computing optimal haplotype pairs...100%|███████████████| Time: 0:01:40
Merging breakpoints...100%|█████████████████████████████| Time: 0:02:43
Writing to file...100%|█████████████████████████████████| Time: 0:00:17


511.827458 seconds (3.04 G allocations: 280.629 GiB, 7.93% gc time)
error overall = 0.005725477913415067 

Imputing typed + untyped SNPs with dynamic programming, width = 2000


Importing genotype file...100%|█████████████████████████| Time: 0:00:05
Importing reference haplotype files...100%|█████████████| Time: 0:03:13
Computing optimal haplotype pairs...100%|███████████████| Time: 0:00:54
Merging breakpoints...100%|█████████████████████████████| Time: 0:06:11
Writing to file...100%|█████████████████████████████████| Time: 0:00:18


669.647724 seconds (3.04 G allocations: 275.428 GiB, 5.92% gc time)
error overall = 0.00858830370003985 



# Beagle 5.0 on incomplete ref panel

In [26]:
# beagle 5
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
function beagle()
    chr = 22
    tgtfile = "target.chr$chr.typedOnly.masked.vcf.gz"
    reffile = "ref.chr$chr.excludeTarget.vcf.gz"
    outfile = "beagle.imputed"
    Base.run(`java -Xmx15g -jar beagle.28Sep18.793.jar gt=$tgtfile ref=$reffile out=$outfile nthreads=4`)
        
    # beagle error rate    
    X_complete = convert_gt(Float32, "target.chr22.full.vcf.gz")
    X_beagle = convert_gt(Float32, "beagle.imputed.vcf.gz")
    n, p = size(X_complete)
    println("error overall = $(sum(X_beagle .!= X_complete) / n / p) \n")
end
beagle()

beagle.28Sep18.793.jar (version 5.0)
Copyright (C) 2014-2018 Brian L. Browning
Enter "java -jar beagle.28Sep18.793.jar" to list command line argument
Start time: 06:01 PM PDT on 24 Apr 2020

Command line: java -Xmx13653m -jar beagle.28Sep18.793.jar
  gt=target.chr22.typedOnly.masked.vcf.gz
  ref=ref.chr22.excludeTarget.vcf.gz
  out=beagle.imputed
  nthreads=4

No genetic map is specified: using 1 cM = 1 Mb

Reference samples:       2,254
Study samples:             250

Window 1 (22:16050115-51244237)
Reference markers:     644,939
Study markers:         100,000

Burnin  iteration 1:           19 seconds
Burnin  iteration 2:           45 seconds
Burnin  iteration 3:           48 seconds
Burnin  iteration 4:           53 seconds
Burnin  iteration 5:           58 seconds
Burnin  iteration 6:           57 seconds

Phasing iteration 1:           55 seconds
Phasing iteration 2:           51 seconds
Phasing iteration 3:           52 seconds
Phasing iteration 4:           51 seconds
Phasing it

# Eagle 2 + Minimac4

In order to use the reference panel in Eagle 2's prephase option, one must first convert it to `.bcf` format via e.g. `htslib` which is *extremely* difficult to install. Even after we went through all the hard work to obtain the final `.bcf` reference file (see commands below), eagle 2.4 STILL SAYS the file is not acceptable (not bgzipped or some processing error). Therefore, I have no choice but to prephase without the reference panel. 

In [ ]:
# THIS CHUNK OF CODE DID NOT WORK
module load htslib/1.9                   # load htslib on hoffman
gunzip ref.chr22.excludeTarget.vcf.gz    # unzip .vcf.gz
bgzip ref.chr22.excludeTarget.vcf        # bgzip the vcf file
bcftools index -f ref.chr22.excludeTarget.vcf.gz   # load some required index, not sure if needed
bcftools view ref.chr22.excludeTarget.vcf > ref.chr22.excludeTarget.bcf # create .bcf -> CREATED FILE BUT says not bgzipped
bgzip ref.chr22.excludeTarget.bcf       # try to bgzip .bcf file -> cannot do so

In [ ]:
# run eagle 2.4: 3367.79 sec on amd-2382 machine (can only run on linux systems)
eagle --vcf=target.chr22.typedOnly.masked.vcf.gz --outPrefix=eagle.phased --numThreads=4 --geneticMapFile=../Eagle_v2.4.1/tables/genetic_map_hg19_withX.txt.gz

In [ ]:
# convert ref file to m3vcf format (Total Run completed in 1 hours, 46 mins, 24 seconds)
/u/home/b/biona001/haplotype_comparisons/Minimac3/bin/Minimac3 --refHaps ref.chr22.excludeTarget.vcf.gz --processReference --prefix ref.chr22.excludeTarget

In [ ]:
# run minimac4 (2619 seconds)
minimac4 --refHaps ref.chr22.excludeTarget.m3vcf.gz --haps eagle.phased.vcf.gz --prefix minimac.imputed.chr22 --format GT --cpus 4

In [3]:
# minimac4 error rate    
X_complete = convert_gt(Float32, "target.chr22.full.vcf.gz")
X_minimac = convert_gt(Float32, "minimac.imputed.chr22.dose.vcf.gz")
n, p = size(X_complete)
println("error overall = $(sum(X_minimac .!= X_complete) / n / p) \n")

error overall = 0.003166693284171061 

